### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [10]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2017M09D14', 'UTC 15:28')


# Settings

In [39]:
MAXTASKS = 250
CANCELTASKS = 0

In [40]:
import pandas as pd
import ee
from retrying import retry
import datetime

In [41]:
ee.Initialize()

# Functions

In [42]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [43]:
taskList = get_tasks()


In [44]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [45]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,error_message,id,output_url,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,10.001850,1505402917041,Asset ingestion: users/rutgerhofste/hybas_lev0...,NaN,ZOVG2E5WGWYKI34LFAL2EBCY,NaN,0.0,0.024867,10.026717,15:28:38,1505402918533,RUNNING,INGEST,1505403518644
1,3.503750,1505402855701,Asset ingestion: users/rutgerhofste/hybas_lev0...,Unable to parse the specified file.,VQLUT2O7RIRUBCSOUCOVMNE6,NaN,NaN,0.045667,3.549417,15:27:38,1505402858441,FAILED,INGEST,1505403068666
2,0.735250,1505355975536,PDomWW_monthY2014M12V15,NaN,IYATZ64AI34774X2RIA7AQTR,https://console.developers.google.com/storage/...,NaN,0.056683,0.791933,02:26:18,1505355978937,COMPLETED,EXPORT_FEATURES,1505356023052
3,0.720000,1505355948909,PDomWW_monthY2014M11V15,NaN,RMWB75TN6V7OOZSZF4TT36WN,https://console.developers.google.com/storage/...,NaN,0.049850,0.769850,02:25:51,1505355951900,COMPLETED,EXPORT_FEATURES,1505355995100
4,0.629350,1505355922649,PDomWW_monthY2014M10V15,NaN,Y374XZNXWDGFFHGBCXGEFVB3,https://console.developers.google.com/storage/...,NaN,0.238483,0.867833,02:25:36,1505355936958,COMPLETED,EXPORT_FEATURES,1505355974719
5,0.836000,1505355896800,PDomWW_monthY2014M09V15,NaN,U2JYSWLFWSFD4O5YVWKZ24MH,https://console.developers.google.com/storage/...,NaN,0.020367,0.856367,02:24:58,1505355898022,COMPLETED,EXPORT_FEATURES,1505355948182
6,1.024317,1505355870620,PDomWW_monthY2014M08V15,NaN,D4SHCUHN2OGBIXLC65MG5FM4,https://console.developers.google.com/storage/...,NaN,0.057633,1.081950,02:24:34,1505355874078,COMPLETED,EXPORT_FEATURES,1505355935537
7,1.175233,1505355844710,PDomWW_monthY2014M07V15,NaN,QSKNFYDOXPA2VUBRWVBM3FFD,https://console.developers.google.com/storage/...,NaN,0.037250,1.212483,02:24:06,1505355846945,COMPLETED,EXPORT_FEATURES,1505355917459
8,0.994117,1505355818545,PDomWW_monthY2014M06V15,NaN,YDJTRVXJUMUWQX7YRBVO3MJT,https://console.developers.google.com/storage/...,NaN,0.179117,1.173233,02:23:49,1505355829292,COMPLETED,EXPORT_FEATURES,1505355888939
9,0.928917,1505355792259,PDomWW_monthY2014M05V15,NaN,I2UHBB7IQXF7UGADOCXR4DFM,https://console.developers.google.com/storage/...,NaN,0.361083,1.290000,02:23:33,1505355813924,COMPLETED,EXPORT_FEATURES,1505355869659


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)